In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holidays
import pytz

from datetime import datetime
from scipy import stats

import eli5
from eli5.sklearn import PermutationImportance
from catboost import *
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.feature_selection import RFE
from scipy.cluster.hierarchy import ward, fcluster
from scipy.spatial.distance import pdist
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from bayes_opt import BayesianOptimization

In [3]:
!pip install holidays eli5 lightgbm bayesian-optimization

  Using cached bayesian_optimization-1.2.0-py3-none-any.whl


In [5]:
train = pd.read_csv('data/train_v1.csv', parse_dates=['date'])
test = pd.read_csv('data/test_v1.csv', parse_dates=['date'])

In [6]:
# drop_cols = ['driver_id', 'day', 'date', 'online_hours', 'holiday', 'next_holiday', 'prev_holiday']
drop_cols = ['driver_id', 'day', 'date', 'online_hours', 'dayofweek']
X_train = train.drop(drop_cols,axis=1)
y_train = train[['online_hours']]
X_test = test.drop(drop_cols,axis=1)
y_test = test[['online_hours']]

In [5]:
# a = X_train.copy()
# a.to_csv('autosmart/data/train/main_train.data', sep='\t', index=False)

# a = (y_train['online_hours'] > 6.5)&(y_train['online_hours'] < 7.5)
# print(np.unique(a, return_counts=True))
# a.to_csv('autosmart/data/train/main_train.solution', index=False)

# a = X_test.copy()
# a.to_csv('autosmart/data/test/main_test.data', sep='\t', index=False)

# a = y_test['online_hours'] == 7
# print(np.unique(a, return_counts=True))
# a.to_csv('autosmart/data/test/main_test.solution', index=False)

In [7]:
ntrain = len(X_train)
ntest = len(X_test)
n_folds = 5

kf = KFold(n_splits = n_folds, random_state=0,shuffle=True)
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
        
def get_oof(clf, x_train, y_train, x_test, y_test=None):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((n_folds, ntest))
    all_train_idx = []
    all_test_idx = []
    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]
        all_train_idx.append(train_index)
        all_test_idx.append(test_index)
        
        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)
        
#         rmse_1_model = np.sqrt(mean_squared_error(clf.predict(x_test).reshape(-1, 1),y_test))
#         print("mse from 1 model only", rmse_1_model)

    oof_test[:] = oof_test_skf.mean(axis=0)
    
#     rmse_oof = np.sqrt(mean_squared_error(oof_test.reshape(-1, 1),y_test))
#     print('average from mean of oof ', rmse_oof)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [8]:
xg_params = {'base_score': 17.46303821732446,
#   'booster': 1.0,
  'gamma': 25.28091906016491,
  'rate_drop': 0.0,
  'reg_alpha': 0.0,
  'reg_lambda': 50.0,
  'scale_pos_weight': 20.36493882748504,
    'learning_rate':0.01,
            'n_estimators':700}

xg_params = {'base_score': 6.501428605913883,
#   'booster': 1.0387339157405941,
  'gamma': 33.94177664699455,
  'rate_drop': 0.21162811600005904,
  'reg_alpha': 13.277332968611312,
  'reg_lambda': 24.578657964016916,
  'scale_pos_weight': 2.668127255854019,
    'learning_rate':0.01,
            'n_estimators':700}

# Extra Trees Parameters
lg_params = {'bagging_fraction': 0.5,
  'lambda_l1': 0.0,
  'lambda_l2': 1.0,
  'max_depth': 7,
  'min_child_samples': 17,
  'min_child_weight': 0.01,
  'num_leaves': 24,
    'learning_rate':0.01,
            'n_estimators':700}

cb_params = {
    'verbose' : 0,
    'learning_rate':0.01,
            'n_estimators':700
}
rf_params = {'max_depth': 10,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 5,
  'min_samples_split': 20,
  'min_weight_fraction_leaf': 0.0}

# Extra Trees Parameters
et_params = {'max_depth': 10,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 5,
  'min_samples_split': 20,
  'min_weight_fraction_leaf': 0.0}

ab_params = {
}

In [9]:
xg = SklearnHelper(clf=XGBRegressor, params=xg_params)
lg = SklearnHelper(clf=LGBMRegressor, params=lg_params)
cb = SklearnHelper(clf=CatBoostRegressor, params=cb_params)
rf = SklearnHelper(clf=RandomForestRegressor, params=rf_params)
et = SklearnHelper(clf=ExtraTreesRegressor, params=et_params)
ab = SklearnHelper(clf=AdaBoostRegressor, params=ab_params)

In [10]:
xg_oof_train, xg_oof_test = get_oof(xg, X_train.values, y_train.values, X_test.values,y_test.values)
print(np.sqrt(mean_squared_error(xg_oof_test,y_test)))
lg_oof_train, lg_oof_test = get_oof(lg, X_train.values, y_train.values, X_test.values,y_test.values) 
print(np.sqrt(mean_squared_error(lg_oof_test,y_test)))
cb_oof_train, cb_oof_test = get_oof(cb, X_train.values, y_train.values, X_test.values,y_test.values) 
print(np.sqrt(mean_squared_error(cb_oof_test,y_test)))
rf_oof_train, rf_oof_test = get_oof(rf, X_train.values, y_train.values, X_test.values,y_test.values)
print(np.sqrt(mean_squared_error(rf_oof_test,y_test)))
et_oof_train, et_oof_test = get_oof(et, X_train.values, y_train.values, X_test.values,y_test.values)
print(np.sqrt(mean_squared_error(et_oof_test,y_test)))
ab_oof_train, ab_oof_test = get_oof(ab, X_train.values, y_train.values, X_test.values,y_test.values)
print(np.sqrt(mean_squared_error(ab_oof_test,y_test)))

[10:50:07] WARNING: ../src/learner.cc:576: 
Parameters: { "rate_drop" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[10:50:25] WARNING: ../src/learner.cc:576: 
Parameters: { "rate_drop" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[10:50:47] WARNING: ../src/learner.cc:576: 
Parameters: { "rate_drop" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but g

/home/user/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/user/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[LightGBM] [Warning] lambda_l1 is set=0.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0


/home/user/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[LightGBM] [Warning] lambda_l1 is set=0.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0


...

1.804705321957365


<ipython-input-7-cb1a9aad3a5a>:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.clf.fit(x_train, y_train)
<ipython-input-7-cb1a9aad3a5a>:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.clf.fit(x_train, y_train)
<ipython-input-7-cb1a9aad3a5a>:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.clf.fit(x_train, y_train)
<ipython-input-7-cb1a9aad3a5a>:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.clf.fit(x_train, y_train)
<ipython-input-7-cb1a9aad3a5a>:12: DataConversionWarning: A column-vector y was passed when a 1d array was e

1.8064405361661537


/home/user/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/user/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/user/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/user/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

1.9580536628290943


In [11]:
X_train = np.concatenate(( xg_oof_train, lg_oof_train, cb_oof_train, rf_oof_train, et_oof_train, ab_oof_train), axis=1)
X_test = np.concatenate(( xg_oof_test, lg_oof_test, cb_oof_test, rf_oof_test, et_oof_test, ab_oof_test), axis=1)

In [12]:
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
rl = Ridge()

In [13]:
rl.fit(X_train,y_train)

Ridge()

In [ ]:
#newest version
np.sqrt(mean_squared_error(y_test,rl.predict(X_test)))

In [12]:
#newest version
np.sqrt(mean_squared_error(y_test,rl.predict(X_test)))

1.803240086763623

In [14]:
#newest version
r2_score(y_test.values.reshape(-1),rl.predict(X_test).reshape(-1))

0.7159207552054878

In [23]:
#newest version
np.sqrt(mean_squared_error(y_test,rl.predict(X_test)))

1.7994981875984477

In [24]:
#newest version
r2_score(y_test.values.reshape(-1),rl.predict(X_test).reshape(-1))

0.7170985162664016

In [11]:
#newest version
np.sqrt(mean_squared_error(y_test,rl.predict(X_test)))

1.8023350734057813

In [13]:
#newest version
r2_score(y_test.values.reshape(-1),rl.predict(X_test).reshape(-1))

0.7162058320483708

In [13]:
#before tuning 1.762724764406399
#after tuning 1.7553690209540496
np.sqrt(mean_squared_error(y_test,rl.predict(X_test)))

1.7553690209540496

In [16]:
#before tuning 0.7037676733623973
#after tuning 0.7067996685215242
r2_score(y_test.values.reshape(-1),rl.predict(X_test).reshape(-1))

0.7067996685215242

In [48]:
# with cluster
np.sqrt(mean_squared_error(y_test,rl.predict(X_test)))

1.7989849833008245

In [49]:
# with cluster
r2_score(y_test.values.reshape(-1),rl.predict(X_test).reshape(-1))

0.717259856305757

In [26]:
# without dayofweek with holiday
np.sqrt(mean_squared_error(y_test,rl.predict(X_test)))

1.755394948895939

In [27]:
# without dayofweek with holiday
r2_score(y_test.values.reshape(-1),rl.predict(X_test).reshape(-1))

0.70679100693845

# Hyperparameter Tuning through Bayesian

## 1. XGBoost 

In [171]:
class xgboost_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train.values
        self.y_train = y_train.values
        self.x_test = x_test.values
        self.y_test = y_test.values
        
    def clean_param(self, param) :
        booster_dict = {1:'gbtree', 2:'gblinear', 3:'dart'}
        params = {'random_state':0, 
                  'subsample':1, 
                  'colsample_bytree':1, 
                  'colsample_bylevel':1}
        
        params['objective'] = 'reg:squarederror'
        params['learning_rate'] = 0.1
        params['n_estimators'] = 100
        params['booster'] = booster_dict[int(param['booster'])]
        params['gamma'] = param['gamma']
        params['reg_alpha'] = param['reg_alpha']
        params['reg_lambda'] = param['reg_lambda']
        params['scale_pos_weight'] = param['scale_pos_weight']
        params['base_score'] = param['base_score']
        params['rate_drop'] = param['rate_drop']
        return params
        
    def evaluate(self,  
                 booster, 
                 gamma, 
                 reg_alpha, 
                 reg_lambda, 
                 scale_pos_weight, 
                 base_score, 
                 rate_drop):

        params = {}
        params['learning_rate'] = 0.1
        params['n_estimators'] = 100
        params['booster'] = booster
        params['gamma'] = gamma
        params['reg_alpha'] = reg_alpha
        params['reg_lambda'] = reg_lambda
        params['scale_pos_weight'] = scale_pos_weight
        params['base_score'] = base_score
        params['rate_drop'] = rate_drop
        params = self.clean_param(params)

        xgb_model = XGBRegressor(**params)
        xgb_model.fit(self.x_train, self.y_train)
        y_pred = xgb_model.predict(self.x_test)
        predictions = [value for value in y_pred]
        mse = mean_squared_error(self.y_test, predictions)
        rmse = np.sqrt(mse)
        return -1*rmse

In [17]:
from sklearn.model_selection import train_test_split
train = pd.read_csv('data/train_v2.csv', parse_dates=['date'])
test = pd.read_csv('data/test_non_anomaly_v2.csv', parse_dates=['date'])
y_train = train[['online_hours']]
X_train = train.drop(['driver_id', 'day', 'date', 'online_hours', 
                      'holiday', 'next_holiday', 'prev_holiday'],axis=1)
X_test = test.drop(['driver_id', 'day', 'date', 'online_hours', 
                      'holiday', 'next_holiday', 'prev_holiday'],axis=1)
y_test = test[['online_hours']]

In [174]:
X_train_subset, X_test_subset, y_train_subset, y_test_subset = train_test_split(X_train, y_train, test_size=0.2)
xt = xgboost_target(X_train_subset, y_train_subset, X_test_subset, y_test_subset)
xgbBO = BayesianOptimization(xt.evaluate, {'booster' : (1, 3),
                                            'gamma' : (0, 50),
                                            'reg_alpha' : (0, 50),
                                            'reg_lambda' : (0, 50),
                                            'scale_pos_weight' : (0, 50),
                                            'base_score' : (0, 50),
                                            'rate_drop': (0, 1)},
                            random_state = 1)

xgbBO.maximize(init_points=15, n_iter=10)

|   iter    |  target   | base_s... |  booster  |   gamma   | rate_drop | reg_alpha | reg_la... | scale_... |
-------------------------------------------------------------------------------------------------------------
|  1        | -3.511    |  20.85    |  2.441    |  0.005719 |  0.3023   |  7.338    |  4.617    |  9.313    |
|  2        | -1.806    |  17.28    |  1.794    |  26.94    |  0.4192   |  34.26    |  10.22    |  43.91    |
|  3        | -3.109    |  1.369    |  2.341    |  20.87    |  0.5587   |  7.019    |  9.905    |  40.04    |
|  4        | -1.825    |  48.41    |  1.627    |  34.62    |  0.8764   |  44.73    |  4.252    |  1.953    |
|  5        | -3.388    |  8.492    |  2.756    |  4.917    |  0.4211   |  47.89    |  26.66    |  34.59    |
|  6        | -3.387    |  15.78    |  2.373    |  41.73    |  0.01829  |  37.51    |  49.44    |  37.41    |
|  7        | -3.387    |  14.02    |  2.579    |  5.161    |  0.4479   |  45.43    |  14.68    |  14.39    |
|  8      

In [176]:
xgbBO.max

{'target': -1.7973869087189835,
 'params': {'base_score': 17.46303821732446,
  'booster': 1.0,
  'gamma': 25.28091906016491,
  'rate_drop': 0.0,
  'reg_alpha': 0.0,
  'reg_lambda': 50.0,
  'scale_pos_weight': 20.36493882748504}}

## 2. LGBM 

In [183]:
class lgbm_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        params = {'boosting_type':'gbdt', 'class_weight':None, 'colsample_bytree':1.0, 
                  'importance_type':'split', 'learning_rate':0.1,
                  'min_child_samples':20, 'min_split_gain':0.0, 'n_estimators':100, 'objective':None,
                  'random_state':0, 'reg_alpha':0.0, 'reg_lambda':0.0, 'silent':True,
                  'subsample':1.0, 'subsample_for_bin':200000, 'subsample_freq':0}
        params['num_leaves'] = int(param['num_leaves'])
        params['min_child_weight'] = int(param['min_child_weight'])
        params['max_depth'] = int(param['max_depth'])
        params['learning_rate'] = 0.1
        params['min_data_in_bin'] = 1
        params['min_data'] = 1
        
        params['min_child_samples'] = int(param['min_child_samples'])
        params['bagging_fraction'] = param['bagging_fraction']
        params['lambda_l1'] = param['lambda_l1']
        params['lambda_l2'] = param['lambda_l2']

        return params
        
    def evaluate(self, min_child_weight, max_depth, num_leaves,
                min_child_samples, bagging_fraction, lambda_l1, lambda_l2):
        params = {'num_leaves':num_leaves, 
                  'min_child_weight':min_child_weight, 
                  'max_depth':max_depth,
                 'min_child_samples':min_child_samples,
                 'bagging_fraction' : bagging_fraction,
                 'lambda_l1' : lambda_l1,
                 'lambda_l2' : lambda_l2}
        
        params = self.clean_param(params)

        lgbm_model = LGBMRegressor(**params)
        lgbm_model.fit(self.x_train, self.y_train)
        y_pred = lgbm_model.predict(self.x_test)
        predictions = [value for value in y_pred]
        mse = mean_squared_error(self.y_test, predictions)
        rmse = np.sqrt(mse)
        return -1*rmse

In [184]:
lt = lgbm_target(X_train_subset, y_train_subset, X_test_subset, y_test_subset)
lgbmBO = BayesianOptimization(lt.evaluate, {'min_child_weight': (0.01, 1),
                                              'max_depth': (1, 7),
                                              'num_leaves': (5, 50),
                                            'min_child_samples' :(10,50),
                                            'bagging_fraction' : (0.5,1),
                                            'lambda_l1' : (0,1),
                                            'lambda_l2' : (0,1)
                                           }, 
                             random_state=3)

lgbmBO.maximize(init_points=15, n_iter=10)

|   iter    |  target   | baggin... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        | -1.784    |  0.7754   |  0.7081   |  0.2909   |  4.065    |  45.72    |  0.8973   |  10.65    |
|  2        | -1.964    |  0.6036   |  0.05147  |  0.4408   |  1.179    |  28.27    |  0.6527   |  17.53    |
|  3        | -1.782    |  0.8381   |  0.5909   |  0.02398  |  4.353    |  20.37    |  0.421    |  17.76    |
|  4        | -1.779    |  0.8466   |  0.4405   |  0.1569   |  4.268    |  41.21    |  0.3133   |  14.99    |
|  5        | -1.776    |  0.694    |  0.9364   |  0.976    |  5.034    |  46.11    |  0.8473   |  22.01    |
|  6        | -1.79     |  0.5461   |  0.6534   |  0.5578   |  3.169    |  19.0     |  0.4125   |  26.1     |
|  7        | -1.777    |  0.6346   |  0.2918   |  0.4577   |  6.163    |  33.45    |  0.2907   |  17.51    |
|  8      

In [224]:
lgbmBO.max

{'target': -1.7744926606010274,
 'params': {'bagging_fraction': 0.5,
  'lambda_l1': 0.0,
  'lambda_l2': 1.0,
  'max_depth': 7.0,
  'min_child_samples': 17.36937547263704,
  'min_child_weight': 0.01,
  'num_leaves': 24.32674024279186}}

## 3. Random Forest 

In [213]:
class rf_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        params = {
            'max_depth' : 5,
            'min_samples_split' : 2,
            'min_samples_leaf': 1,
            'min_weight_fraction_leaf': 0.0,
            'min_impurity_decrease': 0.0
            
        }
        params['max_depth'] = int(param['max_depth'])
        params['min_samples_split'] = int(param['min_samples_split'])
        params['min_samples_leaf'] = int(param['min_samples_leaf'])
        params['min_weight_fraction_leaf'] = param['min_weight_fraction_leaf']
        params['min_impurity_decrease'] = param['min_impurity_decrease']

        return params
        
    def evaluate(self, max_depth, min_samples_split, min_samples_leaf,
                min_weight_fraction_leaf, min_impurity_decrease):
        params = {'max_depth':max_depth, 
                  'min_samples_split':min_samples_split, 
                  'min_samples_leaf':min_samples_leaf,
                 'min_weight_fraction_leaf':min_weight_fraction_leaf,
                 'min_impurity_decrease' : min_impurity_decrease}
        
        params = self.clean_param(params)

        rf_model = RandomForestRegressor(**params)
        rf_model.fit(self.x_train, self.y_train)
        y_pred = rf_model.predict(self.x_test)
        predictions = [value for value in y_pred]
        mse = mean_squared_error(self.y_test, predictions)
        rmse = np.sqrt(mse)
        return -1*rmse

In [216]:
rf = rf_target(X_train_subset, y_train_subset, X_test_subset, y_test_subset)
rfBO = BayesianOptimization(rf.evaluate, {'max_depth': (1, 10),
                                              'min_samples_split': (10, 20),
                                              'min_samples_leaf': (5, 50),
                                            'min_weight_fraction_leaf' :(0,0.5),
                                            'min_impurity_decrease' : (0,1)
                                           }, 
                             random_state=3)

rfBO.maximize(init_points=15, n_iter=10)

|   iter    |  target   | max_depth | min_im... | min_sa... | min_sa... | min_we... |
-------------------------------------------------------------------------------------
|  1        | -2.315    |  5.957    |  0.7081   |  18.09    |  15.11    |  0.4465   |


/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


|  2        | -2.04     |  9.067    |  0.1256   |  14.33    |  10.51    |  0.2204   |
|  3        | -2.262    |  1.269    |  0.4568   |  34.21    |  12.78    |  0.3381   |


/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


|  4        | -2.094    |  6.318    |  0.02398  |  30.15    |  12.59    |  0.2076   |
|  5        | -2.074    |  3.552    |  0.6931   |  24.82    |  11.57    |  0.2723   |


...

|  23       | -1.786    |  10.0     |  0.0      |  5.0      |  20.0     |  0.0      |


/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


|  24       | -2.036    |  10.0     |  1.0      |  5.0      |  10.0     |  0.0      |


/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


|  25       | -1.79     |  10.0     |  0.0      |  36.48    |  20.0     |  0.0      |


In [222]:
rfBO.max

{'target': -1.785930078238171,
 'params': {'max_depth': 10.0,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 5.0,
  'min_samples_split': 20.0,
  'min_weight_fraction_leaf': 0.0}}

## Extra Tree 

In [217]:
class et_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        params = {
            'max_depth' : 5,
            'min_samples_split' : 2,
            'min_samples_leaf': 1,
            'min_weight_fraction_leaf': 0.0,
            'min_impurity_decrease': 0.0
            
        }
        params['max_depth'] = int(param['max_depth'])
        params['min_samples_split'] = int(param['min_samples_split'])
        params['min_samples_leaf'] = int(param['min_samples_leaf'])
        params['min_weight_fraction_leaf'] = param['min_weight_fraction_leaf']
        params['min_impurity_decrease'] = param['min_impurity_decrease']

        return params
        
    def evaluate(self, max_depth, min_samples_split, min_samples_leaf,
                min_weight_fraction_leaf, min_impurity_decrease):
        params = {'max_depth':max_depth, 
                  'min_samples_split':min_samples_split, 
                  'min_samples_leaf':min_samples_leaf,
                 'min_weight_fraction_leaf':min_weight_fraction_leaf,
                 'min_impurity_decrease' : min_impurity_decrease}
        
        params = self.clean_param(params)

        et_model = ExtraTreesRegressor(**params)
        et_model.fit(self.x_train, self.y_train)
        y_pred = et_model.predict(self.x_test)
        predictions = [value for value in y_pred]
        mse = mean_squared_error(self.y_test, predictions)
        rmse = np.sqrt(mse)
        return -1*rmse

In [221]:
et = et_target(X_train_subset, y_train_subset, X_test_subset, y_test_subset)
etBO = BayesianOptimization(et.evaluate, {'max_depth': (1, 10),
                                              'min_samples_split': (10, 20),
                                              'min_samples_leaf': (5, 50),
                                            'min_weight_fraction_leaf' :(0,0.5),
                                            'min_impurity_decrease' : (0,1)
                                           }, 
                             random_state=3)

etBO.maximize(init_points=15, n_iter=10)

|   iter    |  target   | max_depth | min_im... | min_sa... | min_sa... | min_we... |
-------------------------------------------------------------------------------------
|  1        | -2.625    |  5.957    |  0.7081   |  18.09    |  15.11    |  0.4465   |
|  2        | -2.174    |  9.067    |  0.1256   |  14.33    |  10.51    |  0.2204   |
|  3        | -2.412    |  1.269    |  0.4568   |  34.21    |  12.78    |  0.3381   |


/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/tryan/minico

|  4        | -2.3      |  6.318    |  0.02398  |  30.15    |  12.59    |  0.2076   |
|  5        | -2.295    |  3.552    |  0.6931   |  24.82    |  11.57    |  0.2723   |
|  6        | -2.647    |  8.023    |  0.3064   |  14.99    |  13.88    |  0.4682   |
|  7        | -2.237    |  9.784    |  0.6724   |  45.63    |  18.46    |  0.189    |


/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/tryan/minico

|  8        | -2.577    |  1.83     |  0.6534   |  30.1     |  13.62    |  0.1125   |
|  9        | -2.298    |  4.659    |  0.4689   |  17.12    |  12.92    |  0.2288   |
|  10       | -2.244    |  8.745    |  0.5863   |  17.76    |  12.78    |  0.2273   |


...

|  23       | -2.218    |  10.0     |  1.0      |  5.0      |  10.0     |  0.0      |


/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


|  24       | -2.316    |  1.0      |  0.0      |  21.75    |  20.0     |  0.0      |


/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/tryan/miniconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


|  25       | -1.765    |  10.0     |  0.0      |  35.49    |  20.0     |  0.0      |


In [223]:
etBO.max

{'target': -1.7612070508864968,
 'params': {'max_depth': 10.0,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 5.0,
  'min_samples_split': 20.0,
  'min_weight_fraction_leaf': 0.0}}